In [1]:
using CSV
using DataFrames

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201228.csv"));

In [4]:
unique!(rawdata, [:hospital_name, :hospital_pk]);

In [5]:
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :hospital_pk => :hospital_id,
    :state,
    :city => ByRow(x -> ismissing(x) ? x : titlecase(x)) => :city,
    :zip,
    :fips_code,
)
sort!(data, [:hospital, :hospital_id]);

In [6]:
hospitalsystem_rawdata = DataFrame(CSV.File("../rawdata/chsp-hospital-linkage-2018.csv"))
hospitalsystem_dict = Dict(row.ccn => (
    hospital_name = row.hospital_name,
    health_system_id = row.health_sys_id,
    health_system_name = row.health_sys_name,
) for row in eachrow(hospitalsystem_rawdata));

In [7]:
data.system_id = [haskey(hospitalsystem_dict, h) ? hospitalsystem_dict[h].health_system_id : missing for h in data.hospital_id];
data.system_name = [haskey(hospitalsystem_dict, h) ? hospitalsystem_dict[h].health_system_name : missing for h in data.hospital_id];

In [8]:
data

,hospital,hospital_id,state,city,zip,fips_code,system_id,system_name
,String,String,String,String?,Int64?,Int64?,String?,String?
1,Abbeville Area Medical Center,421301,SC,Abbeville,29620,45001,missing,missing
2,Abbeville General Hospital,190034,LA,Abbeville,70510,22113,HSI00000562,Lafayette General Health
3,Abbott Northwestern Hospital,240057,MN,Minneapolis,55407,27053,HSI00000029,Allina Health System
4,Abilene Regional Medical Center,450558,TX,Abilene,79606,48441,HSI00000249,Community Health Systems
5,Abington Memorial Hospital,390231,PA,Abington,19001,42091,HSI00000048,Jefferson Health
6,Abraham Lincoln Memorial Hospital,141322,IL,Lincoln,62656,17107,HSI00000648,Memorial Health System
7,Abrazo Arrowhead Campus,030094,AZ,Glendale,85308,4013,HSI00001066,Tenet Healthcare Corporation
8,Abrazo Central Campus,030030,AZ,Phoenix,85015,4013,HSI00001066,Tenet Healthcare Corporation
9,Abrazo Scottsdale Campus,030083,AZ,Phoenix,85032,4013,HSI00001066,Tenet Healthcare Corporation


In [9]:
data |> CSV.write("../data/hhs_hospital_meta.csv")

"../data/hhs_hospital_meta.csv"